In [ ]:
import pandas as pd
import numpy as np

# Route 66 Travel Time Distributions


In [ ]:
arrival_df = pd.read_csv('MBTA_BusArrivalData.csv', encoding = 'latin-1')

In [ ]:
arrival_df.head()

In [ ]:
to_drop = arrival_df.loc[arrival_df['RouteName']!=66].index

In [ ]:
arrival_df.drop(to_drop, inplace = True)

In [ ]:
arrival_df

In [ ]:
stop_list = ['Harvard Sq @ Garden St - Dawes Island','Harvard Ave @ Commonwealth Ave', 'Washington St @ Walnut','Dudley Station','Washington St @ Pearl St']


In [ ]:
df = arrival_df.loc[arrival_df.StopName.isin(stop_list)].copy()

In [ ]:
df.Vehicle.unique()

In [ ]:
df2.tail()

In [ ]:
df2=df.loc[df.Vehicle == 2215].sort_values(by = 'ScheduledTimeInMin')
tt = df2.ActArrivalTimeInMin.shift(-1)-df2.ActDepartureTimeInMin

In [ ]:
travel_times = {}
for bus in df.Vehicle.unique():
    df2=df.loc[df.Vehicle == bus].sort_values(by = 'ScheduledTimeInMin')
    tt = df2.ActArrivalTimeInMin.shift(-1)-df2.ActDepartureTimeInMin
    for segment in [0, 1, 2, 4, 5, 6]:
        if segment  not in travel_times:
            travel_times[segment] = []
        seg_tt = tt.iloc[segment::8]
        travel_times[segment].extend(seg_tt.loc[seg_tt > 0])


In [ ]:
travel_times

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
for key, values in travel_times.items():
    plt.figure()
    plt.hist(values, density=True)